In [1]:
import random
import numpy as np
import yfinance as yf
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import norm
import math
import scipy.stats as stats
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
tickers=['BLK']

In [3]:
df=yf.download(tickers,start='2017-09-01',end='2022-08-31')

[*********************100%***********************]  1 of 1 completed


In [4]:
df['Return']=df['Close'].pct_change()
df

,Open,High,Low,Close,Adj Close,Volume,Return
Date,,,,,,,
2017-08-31,415.709991,420.279999,415.540009,419.010010,359.001343,501600,NaN
2017-09-01,422.350006,424.279999,420.630005,421.779999,361.374664,441900,0.006611
2017-09-05,419.470001,421.489990,413.040009,413.649994,354.409027,615500,-0.019275
2017-09-06,415.380005,417.700012,411.890015,412.190002,353.158081,430000,-0.003530
2017-09-07,413.750000,414.399994,409.410004,414.250000,354.923096,483300,0.004998
...,...,...,...,...,...,...,...
2022-08-24,694.890015,704.320007,694.010010,700.419983,675.594666,386000,0.004950
2022-08-25,703.690002,711.890015,700.309998,710.599976,685.413757,363700,0.014534
2022-08-26,713.349976,714.270020,675.770020,676.440002,652.464539,633200,-0.048072


In [5]:
sample_size=len(df)
confidence_alpha=0.10
one_minus_alpha=1-confidence_alpha
returns=df['Return'].mean()
Volatility=df['Return'].std()
VaR=returns+norm.ppf(confidence_alpha)*Volatility
VaR

-0.024394300321748677

In [6]:
df['Violation']=[1 if v<VaR else 0 for v in df['Return'].values]
df

,Open,High,Low,Close,Adj Close,Volume,Return,Violation
Date,,,,,,,,
2017-08-31,415.709991,420.279999,415.540009,419.010010,359.001343,501600,NaN,0
2017-09-01,422.350006,424.279999,420.630005,421.779999,361.374664,441900,0.006611,0
2017-09-05,419.470001,421.489990,413.040009,413.649994,354.409027,615500,-0.019275,0
2017-09-06,415.380005,417.700012,411.890015,412.190002,353.158081,430000,-0.003530,0
2017-09-07,413.750000,414.399994,409.410004,414.250000,354.923096,483300,0.004998,0
...,...,...,...,...,...,...,...,...
2022-08-24,694.890015,704.320007,694.010010,700.419983,675.594666,386000,0.004950,0
2022-08-25,703.690002,711.890015,700.309998,710.599976,685.413757,363700,0.014534,0
2022-08-26,713.349976,714.270020,675.770020,676.440002,652.464539,633200,-0.048072,1


In [7]:
proportion_of_violations=df['Violation'].sum()/sample_size
proportion_of_non_violations=1-proportion_of_violations
proportion_of_violations,proportion_of_non_violations

(0.07233704292527822, 0.9276629570747218)

In [8]:
freq_of_non_violations=sample_size-df['Violation'].sum()
freq_of_non_violations,df['Violation'].sum()

(1167, 91)

In [9]:
Kupiec_Chi_sqr=2*np.log((proportion_of_non_violations/one_minus_alpha)**freq_of_non_violations*(proportion_of_violations/confidence_alpha)**df['Violation'].sum())

In [10]:
Kupiec_Chi_sqr

11.721082266040657

In [11]:
stats.chi2.pdf(Kupiec_Chi_sqr,1)

0.00033206685862318986

In [12]:
#VaR at 10% is unreliable